In [1]:
import findspark
findspark.init()
from pyspark.sql import *
from pyspark import  SparkContext

sc = SparkContext( 'local', 'pyspark')
sql = SQLContext(sc)

In [2]:
df = sql.read.csv("train.csv", inferSchema = True, header = True)

In [3]:
df.first()

Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Ticket='A/5 21171', Fare=7.25, Cabin=None, Embarked='S')

In [4]:
df.take(2)

[Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Ticket='A/5 21171', Fare=7.25, Cabin=None, Embarked='S'),
 Row(PassengerId=2, Survived=1, Pclass=1, Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age=38.0, SibSp=1, Parch=0, Ticket='PC 17599', Fare=71.2833, Cabin='C85', Embarked='C')]

In [5]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [6]:
df.describe('Age').show()

+-------+------------------+
|summary|               Age|
+-------+------------------+
|  count|               714|
|   mean| 29.69911764705882|
| stddev|14.526497332334035|
|    min|              0.42|
|    max|              80.0|
+-------+------------------+



In [7]:
df.filter(df['Age'] > 60).count()

22

In [8]:
df.groupby('Sex').count().show()

+------+-----+
|   Sex|count|
+------+-----+
|female|  314|
|  male|  577|
+------+-----+



In [9]:
from pyspark.sql.functions import desc
sortByFare = df.sort((desc('Fare'))).show()
sortByFare = df.sort(-df['Fare']).show() #see the use of - operator for desc

+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+---------------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|  Ticket|    Fare|          Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+---------------+--------+
|        738|       1|     1|Lesurer, Mr. Gust...|  male|35.0|    0|    0|PC 17755|512.3292|           B101|       C|
|        680|       1|     1|Cardeza, Mr. Thom...|  male|36.0|    0|    1|PC 17755|512.3292|    B51 B53 B55|       C|
|        259|       1|     1|    Ward, Miss. Anna|female|35.0|    0|    0|PC 17755|512.3292|           null|       C|
|        342|       1|     1|Fortune, Miss. Al...|female|24.0|    3|    2|   19950|   263.0|    C23 C25 C27|       S|
|        439|       0|     1|   Fortune, Mr. Mark|  male|64.0|    1|    4|   19950|   263.0|    C23 C25 C27|       S|
|         28|       0|     1|Fortune, Mr. Char...|  male

In [10]:
df.registerTempTable('Titanic')

In [11]:
sql.sql('Select Sex, Survived From Titanic').show()

+------+--------+
|   Sex|Survived|
+------+--------+
|  male|       0|
|female|       1|
|female|       1|
|female|       1|
|  male|       0|
|  male|       0|
|  male|       0|
|  male|       0|
|female|       1|
|female|       1|
|female|       1|
|female|       1|
|  male|       0|
|  male|       0|
|female|       0|
|female|       1|
|  male|       0|
|  male|       1|
|female|       0|
|female|       1|
+------+--------+
only showing top 20 rows



In [12]:
sql.sql('Select distinct(Embarked) From Titanic').show()

+--------+
|Embarked|
+--------+
|       Q|
|    null|
|       C|
|       S|
+--------+



In [13]:
sql.sql('Select min(Age) From Titanic').show()

+--------+
|min(Age)|
+--------+
|    0.42|
+--------+



In [14]:
df1 = df.withColumnRenamed('Sex','Gender')
df1.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [15]:
sql.sql('Select Age, Sex From Titanic').sort(df['Age']).distinct().show()

+----+------+
| Age|   Sex|
+----+------+
|null|  male|
|null|female|
|0.42|  male|
|0.67|  male|
|0.75|female|
|0.83|  male|
|0.92|  male|
| 1.0|  male|
| 1.0|female|
| 2.0|  male|
| 2.0|female|
| 3.0|female|
| 3.0|  male|
| 4.0|female|
| 4.0|  male|
| 5.0|female|
| 6.0|female|
| 6.0|  male|
| 7.0|  male|
| 7.0|female|
+----+------+
only showing top 20 rows



In [16]:
df2 = sql.sql('Select Age, Sex, Embarked From Titanic').sort(df['Age']).dropDuplicates(['Age','Sex'])

In [17]:
joined = df2.join(df2, df2.Sex == df2.Sex).show()

+----+------+--------+----+------+--------+
| Age|   Sex|Embarked| Age|   Sex|Embarked|
+----+------+--------+----+------+--------+
|null|female|       C|63.0|female|       S|
|null|female|       C|62.0|female|    null|
|null|female|       C|60.0|female|       C|
|null|female|       C|58.0|female|       S|
|null|female|       C|57.0|female|       S|
|null|female|       C|56.0|female|       C|
|null|female|       C|55.0|female|       S|
|null|female|       C|54.0|female|       C|
|null|female|       C|53.0|female|       S|
|null|female|       C|52.0|female|       C|
|null|female|       C|51.0|female|       S|
|null|female|       C|50.0|female|       C|
|null|female|       C|49.0|female|       C|
|null|female|       C|48.0|female|       C|
|null|female|       C|47.0|female|       S|
|null|female|       C|45.0|female|       S|
|null|female|       C|44.0|female|       C|
|null|female|       C|43.0|female|       S|
|null|female|       C|42.0|female|       C|
|null|female|       C|41.0|femal

# Distinct

In [18]:
sql.sql('SELECT DISTINCT Embarked FROM Titanic').show()

+--------+
|Embarked|
+--------+
|       Q|
|    null|
|       C|
|       S|
+--------+



# Where

In [19]:
sql.sql('SELECT * FROM Titanic WHERE Age BETWEEN 17 and 21').show()
sql.sql('SELECT * FROM Titanic WHERE Age > 17 and Age < 21').show()

+-----------+--------+------+--------------------+------+----+-----+-----+------------------+-------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|            Ticket|   Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+------------------+-------+-----------+--------+
|         13|       0|     3|Saundercock, Mr. ...|  male|20.0|    0|    0|         A/5. 2151|   8.05|       null|       S|
|         28|       0|     1|Fortune, Mr. Char...|  male|19.0|    3|    2|             19950|  263.0|C23 C25 C27|       S|
|         38|       0|     3|Cann, Mr. Ernest ...|  male|21.0|    0|    0|        A./5. 2152|   8.05|       null|       S|
|         39|       0|     3|Vander Planke, Mi...|female|18.0|    2|    0|            345764|   18.0|       null|       S|
|         45|       1|     3|Devaney, Miss. Ma...|female|19.0|    0|    0|            330958| 7.8792|       null|       Q|
|         50|   

# IN

In [20]:
sql.sql('SELECT * FROM Titanic WHERE Age IN (1,80,90)').show()

+-----------+--------+------+--------------------+------+----+-----+-----+---------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|         Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------------+-------+-----+--------+
|        165|       0|     3|Panula, Master. E...|  male| 1.0|    4|    1|        3101295|39.6875| null|       S|
|        173|       1|     3|Johnson, Miss. El...|female| 1.0|    1|    1|         347742|11.1333| null|       S|
|        184|       1|     2|Becker, Master. R...|  male| 1.0|    2|    1|         230136|   39.0|   F4|       S|
|        382|       1|     3|"Nakid, Miss. Mar...|female| 1.0|    0|    2|           2653|15.7417| null|       C|
|        387|       0|     3|Goodwin, Master. ...|  male| 1.0|    5|    2|        CA 2144|   46.9| null|       S|
|        631|       1|     1|Barkworth, Mr. Al...|  male|80.0|    0|    0|          2704

# Like

In [21]:
sql.sql("SELECT * FROM Titanic WHERE Name Like 'Va%' ").show()

+-----------+--------+------+--------------------+------+----+-----+-----+------+-----+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|Ticket| Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+------+-----+-----+--------+
|         19|       0|     3|Vander Planke, Mr...|female|31.0|    1|    0|345763| 18.0| null|       S|
|         39|       0|     3|Vander Planke, Mi...|female|18.0|    2|    0|345764| 18.0| null|       S|
|        171|       0|     1|Van der hoef, Mr....|  male|61.0|    0|    0|111240| 33.5|  B19|       S|
|        201|       0|     3|Vande Walle, Mr. ...|  male|28.0|    0|    0|345770|  9.5| null|       S|
|        334|       0|     3|Vander Planke, Mr...|  male|16.0|    2|    0|345764| 18.0| null|       S|
|        356|       0|     3|Vanden Steen, Mr....|  male|28.0|    0|    0|345783|  9.5| null|       S|
|        420|       0|     3|Van Impe, Miss. C...|female|10.0|    0|    2

In [22]:
sql.sql("SELECT * FROM Titanic WHERE Name Like '%z%' ").show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------+-------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|    Ticket|   Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------+-------+-----------+--------+
|         12|       1|     1|Bonnell, Miss. El...|female|58.0|    0|    0|    113783|  26.55|       C103|       S|
|         54|       1|     2|Faunthorpe, Mrs. ...|female|29.0|    1|    0|      2926|   26.0|       null|       S|
|         67|       1|     2|Nye, Mrs. (Elizab...|female|29.0|    0|    0|C.A. 29395|   10.5|        F33|       S|
|         70|       0|     3|   Kink, Mr. Vincenz|  male|26.0|    2|    0|    315151| 8.6625|       null|       S|
|         80|       1|     3|Dowdell, Miss. El...|female|30.0|    0|    0|    364516| 12.475|       null|       S|
|        131|       0|     3|Drazenoic, Mr. Jozef|  male|33.0|    0|    0|    34

# Wild Cards
%   -> zero or more characters
_   -> exactly one character
[]  -> Any character in bracket
[^] -> Any character except in bracket

In [23]:
sql.sql("SELECT * FROM Titanic WHERE Name Like '[A]' ").show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



# Order By

In [24]:
sql.sql("SELECT * FROM Titanic ORDER BY Fare DESC").show()

+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+---------------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|  Ticket|    Fare|          Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+---------------+--------+
|        738|       1|     1|Lesurer, Mr. Gust...|  male|35.0|    0|    0|PC 17755|512.3292|           B101|       C|
|        680|       1|     1|Cardeza, Mr. Thom...|  male|36.0|    0|    1|PC 17755|512.3292|    B51 B53 B55|       C|
|        259|       1|     1|    Ward, Miss. Anna|female|35.0|    0|    0|PC 17755|512.3292|           null|       C|
|        342|       1|     1|Fortune, Miss. Al...|female|24.0|    3|    2|   19950|   263.0|    C23 C25 C27|       S|
|        439|       0|     1|   Fortune, Mr. Mark|  male|64.0|    1|    4|   19950|   263.0|    C23 C25 C27|       S|
|         28|       0|     1|Fortune, Mr. Char...|  male

# Limit

In [25]:
sql.sql("SELECT * FROM Titanic ORDER BY Fare DESC LIMIT 2").show()

+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|  Ticket|    Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+
|        259|       1|     1|    Ward, Miss. Anna|female|35.0|    0|    0|PC 17755|512.3292|       null|       C|
|        680|       1|     1|Cardeza, Mr. Thom...|  male|36.0|    0|    1|PC 17755|512.3292|B51 B53 B55|       C|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+



# Group By

In [26]:
sql.sql("SELECT Sex, Embarked, AVG(Age) FROM Titanic GROUP BY Sex, Embarked").show()

+------+--------+------------------+
|   Sex|Embarked|          avg(Age)|
+------+--------+------------------+
|  male|       C| 32.99884057971015|
|  male|       S|30.291440217391305|
|female|       Q|24.291666666666668|
|female|    null|              50.0|
|female|       S|27.771505376344088|
|  male|       Q|           30.9375|
|female|       C| 28.34426229508197|
+------+--------+------------------+



In [27]:
sql.sql("SELECT Sex, AVG(Age) AS Avg FROM Titanic GROUP BY Sex, Embarked ORDER BY Avg").show()

+------+------------------+
|   Sex|               Avg|
+------+------------------+
|female|24.291666666666668|
|female|27.771505376344088|
|female| 28.34426229508197|
|  male|30.291440217391305|
|  male|           30.9375|
|  male| 32.99884057971015|
|female|              50.0|
+------+------------------+



In [28]:
sql.sql("SELECT Sex, Embarked, AVG(Age), Count(*) FROM Titanic GROUP BY Sex, Embarked").show()

+------+--------+------------------+--------+
|   Sex|Embarked|          avg(Age)|count(1)|
+------+--------+------------------+--------+
|  male|       C| 32.99884057971015|      95|
|  male|       S|30.291440217391305|     441|
|female|       Q|24.291666666666668|      36|
|female|    null|              50.0|       2|
|female|       S|27.771505376344088|     203|
|  male|       Q|           30.9375|      41|
|female|       C| 28.34426229508197|      73|
+------+--------+------------------+--------+



# HAVING
The HAVING clause was added to SQL because the WHERE keyword could not be used with aggregate functions.

In [29]:
sql.sql("SELECT Embarked, AVG(Age), COUNT(*) FROM Titanic GROUP BY Embarked HAVING COUNT(Embarked) > 100").show()

+--------+-----------------+--------+
|Embarked|         avg(Age)|count(1)|
+--------+-----------------+--------+
|       C|30.81476923076923|     168|
|       S|29.44539711191336|     644|
+--------+-----------------+--------+



Performance wise Having is slower than Where because where first filters rows and then groups

In [30]:
sql.sql("SELECT Embarked, AVG(Age) FROM Titanic WHERE Embarked IN ('C','S') GROUP BY Embarked").show()

sql.sql("SELECT Embarked, AVG(Age) FROM Titanic GROUP BY Embarked HAVING Embarked IN ('C','S')").show()

+--------+-----------------+
|Embarked|         avg(Age)|
+--------+-----------------+
|       C|30.81476923076923|
|       S|29.44539711191336|
+--------+-----------------+

+--------+-----------------+
|Embarked|         avg(Age)|
+--------+-----------------+
|       C|30.81476923076923|
|       S|29.44539711191336|
+--------+-----------------+



# OVER

Needed because we want aggregated and non aaggregated data

In [31]:
sql.sql('SELECT Embarked, Sex, COUNT(Sex) OVER(PARTITION BY Sex) FROM Titanic').show()

+--------+------+-------------------------------------------------------------------------------------------+
|Embarked|   Sex|count(Sex) OVER (PARTITION BY Sex ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)|
+--------+------+-------------------------------------------------------------------------------------------+
|       C|female|                                                                                        314|
|       S|female|                                                                                        314|
|       S|female|                                                                                        314|
|       S|female|                                                                                        314|
|       C|female|                                                                                        314|
|       S|female|                                                                                        314|
|       S|

In [32]:
sql.sql('SELECT AVG(Age) OVER() AS AvgAge, Name FROM Titanic').show()

+-----------------+--------------------+
|           AvgAge|                Name|
+-----------------+--------------------+
|29.69911764705882|Braund, Mr. Owen ...|
|29.69911764705882|Cumings, Mrs. Joh...|
|29.69911764705882|Heikkinen, Miss. ...|
|29.69911764705882|Futrelle, Mrs. Ja...|
|29.69911764705882|Allen, Mr. Willia...|
|29.69911764705882|    Moran, Mr. James|
|29.69911764705882|McCarthy, Mr. Tim...|
|29.69911764705882|Palsson, Master. ...|
|29.69911764705882|Johnson, Mrs. Osc...|
|29.69911764705882|Nasser, Mrs. Nich...|
|29.69911764705882|Sandstrom, Miss. ...|
|29.69911764705882|Bonnell, Miss. El...|
|29.69911764705882|Saundercock, Mr. ...|
|29.69911764705882|Andersson, Mr. An...|
|29.69911764705882|Vestrom, Miss. Hu...|
|29.69911764705882|Hewlett, Mrs. (Ma...|
|29.69911764705882|Rice, Master. Eugene|
|29.69911764705882|Williams, Mr. Cha...|
|29.69911764705882|Vander Planke, Mr...|
|29.69911764705882|Masselmani, Mrs. ...|
+-----------------+--------------------+
only showing top

In [33]:
sql.sql('SELECT AVG(Age) OVER() AS AvgAge, Count(Embarked) OVER() AS EmbCount, * FROM Titanic').show()

+-----------------+--------+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|           AvgAge|EmbCount|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------------+--------+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|29.69911764705882|     889|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|29.69911764705882|     889|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|29.69911764705882|     889|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|29.69911764705882|     889|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|29.69

In [34]:
sql.sql('SELECT AVG(Age) OVER() AS AvgAge, AVG(Age) OVER() - Age AS AgeDiff, Name FROM Titanic').show()

+-----------------+------------------+--------------------+
|           AvgAge|           AgeDiff|                Name|
+-----------------+------------------+--------------------+
|29.69911764705882|  7.69911764705882|Braund, Mr. Owen ...|
|29.69911764705882| -8.30088235294118|Cumings, Mrs. Joh...|
|29.69911764705882|  3.69911764705882|Heikkinen, Miss. ...|
|29.69911764705882| -5.30088235294118|Futrelle, Mrs. Ja...|
|29.69911764705882| -5.30088235294118|Allen, Mr. Willia...|
|29.69911764705882|              null|    Moran, Mr. James|
|29.69911764705882|-24.30088235294118|McCarthy, Mr. Tim...|
|29.69911764705882| 27.69911764705882|Palsson, Master. ...|
|29.69911764705882|  2.69911764705882|Johnson, Mrs. Osc...|
|29.69911764705882| 15.69911764705882|Nasser, Mrs. Nich...|
|29.69911764705882| 25.69911764705882|Sandstrom, Miss. ...|
|29.69911764705882|-28.30088235294118|Bonnell, Miss. El...|
|29.69911764705882|  9.69911764705882|Saundercock, Mr. ...|
|29.69911764705882| -9.30088235294118|An

In [35]:
sql.sql('SELECT DISTINCT Embarked, COUNT(1) OVER(PARTITION BY Embarked) AS NUM, COUNT(1) OVER() AS TotalCount FROM Titanic').show()

+--------+---+----------+
|Embarked|NUM|TotalCount|
+--------+---+----------+
|    null|  2|       891|
|       C|168|       891|
|       Q| 77|       891|
|       S|644|       891|
+--------+---+----------+



In [36]:
sql.sql('SELECT PassengerId, SUM(Fare) OVER(ORDER BY PassengerId) AS FareCumSum, COUNT(1) OVER() AS TotalCount FROM Titanic').show()

+-----------+------------------+----------+
|PassengerId|        FareCumSum|TotalCount|
+-----------+------------------+----------+
|          1|              7.25|       891|
|          2|           78.5333|       891|
|          3|           86.4583|       891|
|          4|          139.5583|       891|
|          5|          147.6083|       891|
|          6|156.06660000000002|       891|
|          7|          207.9291|       891|
|          8|          229.0041|       891|
|          9|240.13739999999999|       891|
|         10|          270.2082|       891|
|         11|286.90819999999997|       891|
|         12|          313.4582|       891|
|         13|          321.5082|       891|
|         14|352.78319999999997|       891|
|         15|360.63739999999996|       891|
|         16|376.63739999999996|       891|
|         17|405.76239999999996|       891|
|         18|418.76239999999996|       891|
|         19|436.76239999999996|       891|
|         20|          443.9874|

# JOIN

In [50]:
from pyspark.sql import Row

l = [(1,25),(2,22),(3,20),(4,26)]
rdd = sc.parallelize(l)
people = rdd.map(lambda x: Row(id=x[0], age=int(x[1])))
leftDF = sql.createDataFrame(people)
leftDF.registerTempTable('leftDF')
leftDF.show()

+---+---+
|age| id|
+---+---+
| 25|  1|
| 22|  2|
| 20|  3|
| 26|  4|
+---+---+



In [51]:
l = [(2,100000),(3,95000),(4,120000),(5,110000)]
rdd = sc.parallelize(l)
people = rdd.map(lambda x: Row(id=x[0], salary=int(x[1])))
rightDF = sql.createDataFrame(people)
rightDF.registerTempTable('rightDF')
rightDF.show()

+---+------+
| id|salary|
+---+------+
|  2|100000|
|  3| 95000|
|  4|120000|
|  5|110000|
+---+------+



# Inner Join

In [43]:
sql.sql('SELECT * FROM leftDF INNER JOIN rightDF ON leftDF.id = rightDF.id').show()

+---+---+---+------+
|age| id| id|salary|
+---+---+---+------+
| 20|  3|  3| 95000|
| 22|  2|  2|100000|
| 26|  4|  4|120000|
+---+---+---+------+



# Left Outer Join

In [44]:
sql.sql('SELECT * FROM leftDF LEFT OUTER JOIN rightDF ON leftDF.id = rightDF.id').show()

+---+---+----+------+
|age| id|  id|salary|
+---+---+----+------+
| 25|  1|null|  null|
| 20|  3|   3| 95000|
| 22|  2|   2|100000|
| 26|  4|   4|120000|
+---+---+----+------+



# Right Outer Join

In [45]:
sql.sql('SELECT * FROM leftDF RIGHT OUTER JOIN rightDF ON leftDF.id = rightDF.id').show()

+----+----+---+------+
| age|  id| id|salary|
+----+----+---+------+
|null|null|  5|110000|
|  20|   3|  3| 95000|
|  22|   2|  2|100000|
|  26|   4|  4|120000|
+----+----+---+------+



# Full Outer Join

In [46]:
sql.sql('SELECT * FROM leftDF FULL OUTER JOIN rightDF ON leftDF.id = rightDF.id').show()

+----+----+----+------+
| age|  id|  id|salary|
+----+----+----+------+
|null|null|   5|110000|
|  25|   1|null|  null|
|  20|   3|   3| 95000|
|  22|   2|   2|100000|
|  26|   4|   4|120000|
+----+----+----+------+



# Cross Join

In [47]:
sql.sql('SELECT * FROM leftDF CROSS JOIN rightDF').show()

+---+---+---+------+
|age| id| id|salary|
+---+---+---+------+
| 25|  1|  2|100000|
| 25|  1|  3| 95000|
| 25|  1|  4|120000|
| 25|  1|  5|110000|
| 22|  2|  2|100000|
| 22|  2|  3| 95000|
| 22|  2|  4|120000|
| 22|  2|  5|110000|
| 20|  3|  2|100000|
| 20|  3|  3| 95000|
| 20|  3|  4|120000|
| 20|  3|  5|110000|
| 26|  4|  2|100000|
| 26|  4|  3| 95000|
| 26|  4|  4|120000|
| 26|  4|  5|110000|
+---+---+---+------+



# Self Join

In [49]:
sql.sql('SELECT * FROM leftDF a, leftDF b WHERE a.age > b.age').show()

+---+---+---+---+
|age| id|age| id|
+---+---+---+---+
| 25|  1| 22|  2|
| 25|  1| 20|  3|
| 22|  2| 20|  3|
| 26|  4| 25|  1|
| 26|  4| 22|  2|
| 26|  4| 20|  3|
+---+---+---+---+

